In [1]:
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import kstest
import plotly.express as px
import plotly.graph_objects as go
import openpyxl
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
import time 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

SEED = 42



In [2]:
best = pd.read_csv('C:/Users/eddem/chippin/cust_best_sample.csv') 
cust = pd.read_csv('C:/Users/eddem/Chippin/cust_sample.csv') 
trx = pd.read_csv('C:/Users/eddem/Chippin/trx_sample.csv') 

In [3]:
merged_trx = pd.merge(trx, cust, on='cb_customer_id', how='inner')
df = pd.merge(merged_trx, best, on='unique_customer_id', how='left')
df.drop_duplicates(inplace=True)
df_copy = df.copy()
df_copy.head()

,cb_customer_id,transaction_date,amount_after_discount,cb_branch_id,amount_before_discount,amount_discount,unique_customer_id,gender,date_of_birth
0,6821860387409851780,2016-07-26,-0.327119,90,-0.411608,-0.443104,14833856522183602191,UNKNOWN,NaN
1,6821860387409851780,2016-08-02,-0.327119,90,-0.411608,-0.443104,14833856522183602191,UNKNOWN,NaN
2,3372770238413923649,2016-06-20,-0.389015,26,-0.331083,-0.281383,219680563978627406,UNKNOWN,NaN
3,3220055865641973068,2016-06-15,-0.344771,90,-0.447712,-0.488551,13102778631438780637,UNKNOWN,NaN
4,1603206957613929399,2016-06-20,-1.139010,33,-1.580552,-1.771293,13698985552782800647,UNKNOWN,NaN


In [4]:
# UNKNOWN tanımlı satırları na olarak atadım.
df_copy['gender'] = df_copy['gender'].replace('UNKNOWN', np.nan)
df_copy.drop(columns=['date_of_birth',"gender"], inplace=True)

In [10]:
print("#### NULL değerler  ###")
print(df_copy.isnull().sum())  

#### NULL değerler  ###
cb_customer_id                 0
transaction_date               0
amount_after_discount          0
cb_branch_id                   0
amount_before_discount         0
amount_discount                0
unique_customer_id             0
gender                    148465
date_of_birth             149917
dtype: int64


In [6]:
# Data tipini tarih olarak düzeltim
df_copy['transaction_date'] = pd.to_datetime(df_copy['transaction_date'])


# Feature Engineering

In [7]:
# uğradıkları şube sayısının eklenmesi:
df_copy['branch_count'] = df_copy.groupby('unique_customer_id')['cb_branch_id'].transform('nunique')
# indirimlerin toplanması
df_copy['discount_sum'] = df_copy.groupby('unique_customer_id')['amount_discount'].transform('sum')
# indirim öncesi harcamalarının toplanması
df_copy['Monetary_beforeDis'] = df_copy.groupby('unique_customer_id')['amount_before_discount'].transform('sum')

## RFM / Değişken Üretme


In [8]:
# Datasetindeki maksimum tarihi almak için yaptım.
max_date = df_copy['transaction_date'].max() 

# Recency icin;
rfm_data = df_copy.groupby('unique_customer_id').agg({
    'transaction_date': lambda x: x.max(), 
})

rfm_data['Recency'] = (max_date - rfm_data['transaction_date']).dt.days
rfm_data.drop(columns='transaction_date', inplace=True)

# Frequency icin:
freq_data = df_copy.groupby('unique_customer_id').agg({
    'transaction_date': 'count'
}).rename(columns={'transaction_date': 'Frequency'})

# Monetary icin:
mon_data = df_copy.groupby('unique_customer_id').agg({
    'amount_after_discount': 'sum'
}).rename(columns={'amount_after_discount': 'Monetary'})

# Joinleme
rfm = rfm_data.join(freq_data).join(mon_data)

##### Skorları Atamak İcin;
## Recency Skoru icin;
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
## Frequency Skoru İcin;
bins = [0, 1, 2, 3, 5, np.inf]
labels = [1, 2, 3, 4, 5]
rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=bins, labels=labels)
## Monetary Skoru İcin;
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])

## Skorların birlestirilmesi
rfm['RF_Score'] = rfm['R_Score'].astype(str) + \
                   rfm['F_Score'].astype(str) 

## Skorlara uydun segmentleri atamak icin;
seg_map = {
    r'[1-2][1-2]': 'Churn',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['RF_segment'] = rfm['RF_Score'].replace(seg_map, regex=True)


In [9]:
# rfm ve df_copy birleştirmek icin;
data = pd.merge(df_copy, rfm, on='unique_customer_id', how='left')


In [10]:
# Modellerde denemek icin ürettigim diger degiskenler
data['avg_amount_spent'] = data['Monetary'] / data['Frequency']
data['avg_discount'] = data['discount_sum'] / data['Frequency']
data["discount_rate"] = data['discount_sum'] / data['Monetary_beforeDis']
data["discount_per_branch"]  =  data['discount_sum']/  data['branch_count']
data["Monetary_branch"]  =  data['discount_sum']/  data['branch_count']
data['branch_discount_interaction'] = data['branch_count'] * data['discount_rate']
data['spend_ratio'] = data['Monetary'] / data['Monetary_beforeDis']
data['discount_impact'] = 1 - data['spend_ratio']

# Churn modeli

In [11]:
# RF segmentindeki Churn olanlar  hedef değişkene atanmıştır. 
data['churn_flag'] = np.where(data['RF_segment'] == "Churn", 1, 0)

# Gereksiz kolonları atmak için.
data_new = data[['branch_count','discount_sum',  'Recency','Frequency', ## Recency ve Frequecy CLV predictionda kullanılacaktır.
                  "unique_customer_id", "discount_rate","discount_per_branch",
                  'Monetary_beforeDis','Monetary','avg_amount_spent','avg_discount', 'churn_flag',
                    "Monetary_branch", 'branch_discount_interaction','spend_ratio','discount_impact']]
data_new.drop_duplicates(inplace=True)

In [12]:
# recency ve frequency data sızıntısı olmaması için çıkarılmıştır.

x = data_new[[  'branch_count','discount_sum',"unique_customer_id",  "discount_rate","discount_per_branch",
         'Monetary_beforeDis','Monetary' ,'avg_amount_spent','avg_discount','branch_discount_interaction',
         'spend_ratio','discount_impact'
]]

y = data_new[['churn_flag']]

x.drop_duplicates(inplace=True)

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=SEED, stratify=y, 
    shuffle=True
)

print("Train set sınıf dağılımı:\n", y_train.value_counts())
print("\nTest set sınıf dağılımı:\n", y_test.value_counts())


Train set sınıf dağılımı:
 churn_flag
0             65659
1             40800
dtype: int64

Test set sınıf dağılımı:
 churn_flag
0             16415
1             10200
dtype: int64


In [14]:
Train_val = pd.concat([x_train,  y_train], axis=1)
val =  pd.concat([x_test,  y_test], axis=1)

In [15]:
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('model', XGBClassifier(random_state=42))  
])

pipeline.fit(x_train,  y_train)

y_pred = pipeline.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Seti Doğruluk Skoru:", accuracy)
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))

# Scaler: Standard Scaler, Model: XGBClassifier


Test Seti Doğruluk Skoru: 0.704903250046966

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76     16415
           1       0.62      0.59      0.61     10200

    accuracy                           0.70     26615
   macro avg       0.69      0.68      0.69     26615
weighted avg       0.70      0.70      0.70     26615



# CLV Prediction

In [16]:
CLD = data_new.copy()


In [17]:
CLD["CLV"]  = CLD['Monetary']

CLD.drop_duplicates(inplace=True)

In [18]:
features = [ "Recency",'Frequency', 'avg_discount', 'branch_count']

In [19]:
x = CLD[features]
y = CLD["CLV"]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=SEED ,
    shuffle=True
)


In [21]:

# 12. Özellik Ölçeklendirme (Özellikle regresyon modellerinde faydalıdır)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# 13. Model Eğitimi: Örneğin RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(x_train, y_train)

# 14. Model Değerlendirmesi
y_pred = rf_reg.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


# Adjusted R² hesaplanması:
n = len(y_test)               # Test setindeki örnek sayısı
p = X_test_scaled.shape[1]    # Kullanılan özellik sayısı
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print("Test RMSE:", rmse)
print("Test R^2 Score:", r2)
print("Test Adjusted R^2 Score:", adjusted_r2)


Test RMSE: 0.43362311880544013
Test R^2 Score: 0.8180214935904436
Test Adjusted R^2 Score: 0.8179941386853088
